In [49]:
import os
import requests

video_path = "data/example/input_video.mp4"
model = "yolo"
interesting_frames = [212, 400, 460]

images_for_paper_path = "data/example/images_for_paper"

In [50]:
# Test if service is running
response = requests.get("http://localhost:4321/")
print(response.json())

{'message': 'Football Referee Evaluation API is running'}


In [51]:
def make_post_request(endpoint: str, body: dict) -> dict:
    response = requests.post(f"http://localhost:4321/{endpoint}", json=body)
    if response.status_code != 200:
        msg = f"Request failed with status code {response.status_code}: {response.text}"
        raise Exception(msg)
    return response.json()

In [52]:
# Track players
tracking_results_path = "data/example/predictions/tracking_results"
# tracking_results = make_post_request(
#     endpoint="track", 
#     body = {
#         "video_path": video_path,
#         "model": model,
#         "results_path": tracking_results_path
#     }
# )

In [53]:
# Load the detections file
import json
with open(os.path.join(tracking_results_path, "detections.json"), "r") as f:
    detections = json.load(f)

In [54]:
# Load the video and extract the interesting frames
import cv2
import numpy as np
import os

# Load the video
cap = cv2.VideoCapture(video_path)

# Create output directory if it doesn't exist
os.makedirs(images_for_paper_path, exist_ok=True)

# Process each interesting frame
for interesting_frame in interesting_frames:
    # Set the frame position to the interesting frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, interesting_frame)
    
    # Read the frame
    ret, frame = cap.read()
    if not ret:
        raise Exception(f"Could not read frame {interesting_frame}")
    
    # Get detections for this specific frame
    frame_detections = next((item for item in detections if item["frame_id"] == interesting_frame), None)
    
    if frame_detections is None:
        print(f"No detections found for frame {interesting_frame}")
        continue
    
    # Create a copy of the frame to draw on
    annotated_frame = frame.copy()
    
    # Draw bounding boxes and IDs
    for detection in frame_detections["detections"]:
        # Extract bbox coordinates
        bbox = detection["bbox"]
        x1, y1, x2, y2 = bbox["x1"], bbox["y1"], bbox["x2"], bbox["y2"]
        track_id = detection["track_id"]
        
        # Draw bright green bbox
        color = (0, 255, 0)  # Bright green in BGR
        thickness = 2
        cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
        
        # Add track ID text
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(annotated_frame, f"{track_id}", (int(x1), int(y1) - 10), 
                    font, 0.7, color, 2)
        
        # Draw a black circle at the player's feet (bottom center of bounding box)
        feet_x = int((x1 + x2) / 2)  # Center x-coordinate
        feet_y = int(y2)  # Bottom y-coordinate
        circle_color = (0, 0, 0)  # Black in BGR
        circle_radius = 12
        circle_thickness = -1  # Filled circle
        cv2.circle(annotated_frame, (feet_x, feet_y), circle_radius, circle_color, circle_thickness)
    
    # Save the annotated frame as a PNG image
    output_path = f"{images_for_paper_path}/tracking_frame_{interesting_frame}.png"
    cv2.imwrite(output_path, annotated_frame)
    
    print(f"Saved annotated frame {interesting_frame} to {output_path}")

# Release the video capture
cap.release()

Saved annotated frame 212 to data/example/images_for_paper/tracking_frame_212.png
Saved annotated frame 400 to data/example/images_for_paper/tracking_frame_400.png
Saved annotated frame 460 to data/example/images_for_paper/tracking_frame_460.png


In [55]:
# Perspective transformation
transform_coordinates_results_path = "data/example/predictions/coordinate_transformation"
# transform_coordinates_results = make_post_request(
#     endpoint="transform-coordinates", 
#     body = {
#         "video_path": video_path,
#         "detections": detections,
#         "results_path": transform_coordinates_results_path
#     }
# )

In [56]:
# Load the detections file
import json
with open(os.path.join(transform_coordinates_results_path, "detections.json"), "r") as f:
    detections = json.load(f)
    
warped_image_paths = [f"data/example/predictions/coordinate_transformation/warped_images/frame_{interesting_frame:06d}.jpg" for interesting_frame in interesting_frames]

In [57]:
# Load the warped images and the pitch background
pitch_background = cv2.imread("src/utils/pitch_2.png")

if pitch_background is None:
    print(f"Error: Could not load pitch background from pitch_2.png")
else:
    # Get pitch background dimensions
    pitch_height, pitch_width = pitch_background.shape[:2]
    
    for i, warped_image_path in enumerate(warped_image_paths):
        warped_image = cv2.imread(warped_image_path)
        
        if warped_image is None:
            print(f"Error: Could not load warped image from {warped_image_path}")
            continue
            
        # Resize warped image to match pitch background dimensions
        resized_warped_image = cv2.resize(warped_image, (pitch_width, pitch_height))
        
        # Create a semi-transparent overlay
        alpha = 0.6  # Transparency factor (0.0 = fully transparent, 1.0 = fully opaque)
        overlay = pitch_background.copy()
        
        # Overlay the warped image on the pitch background
        cv2.addWeighted(resized_warped_image, alpha, pitch_background, 1 - alpha, 0, overlay)
        
        # Create a copy for annotation
        annotated_image = overlay.copy()
        
        # Find the detections for the current interesting frame
        current_frame = interesting_frames[i]
        frame_detections = None
        for frame_data in detections:
            if frame_data["frame_id"] == current_frame:
                frame_detections = frame_data["detections"]
                break
        
        if frame_detections:
            # Draw black dots at player positions
            for detection in frame_detections:
                if "minimap_coordinates" in detection and detection["minimap_coordinates"]:
                    coords = detection["minimap_coordinates"]
                    
                    # Rescale coordinates using x_max and y_max to match pitch dimensions
                    x_max = coords["x_max"]
                    y_max = coords["y_max"]
                    x_scaled = int((coords["x"] / x_max) * pitch_width)
                    y_scaled = int((coords["y"] / y_max) * pitch_height)
                    
                    # Draw a filled black circle (dot)
                    cv2.circle(annotated_image, (x_scaled, y_scaled), 10, (0, 0, 0), -1)  # -1 means filled circle
                    
        # Save the annotated image
        warped_output_path = f"{images_for_paper_path}/warped_tracking_{current_frame:06d}.png"
        cv2.imwrite(warped_output_path, annotated_image)
        print(f"Saved annotated warped image to {warped_output_path}")


Saved annotated warped image to data/example/images_for_paper/warped_tracking_000212.png
Saved annotated warped image to data/example/images_for_paper/warped_tracking_000400.png
Saved annotated warped image to data/example/images_for_paper/warped_tracking_000460.png


In [58]:
# Assign colors to players
color_assignment_results_path = "data/example/predictions/color_assignment"
# color_assignment_results = make_post_request(
#     endpoint="assign-colors", 
#     body = {
#         "video_path": video_path,
#         "detections": detections,
#         "results_path": color_assignment_results_path
#     }
# )

In [59]:
with open(os.path.join(color_assignment_results_path, "detections.json"), "r") as f:
    detections = json.load(f)

In [60]:
# Assign roles
role_assignment_results_path = "data/example/predictions/role_assignment"
# role_assignment_results = make_post_request(
#     endpoint="assign-roles", 
#     body = {
#         "video_path": video_path,
#         "detections": detections,
#         "results_path": role_assignment_results_path
#     }
# )

In [61]:
with open(os.path.join(role_assignment_results_path, "detections.json"), "r") as f:
    detections = json.load(f)

In [79]:
# Visualize players on the pitch with team jersey colors and roles for all interesting frames
import numpy as np

# Load the soccer pitch
pitch_img_original = cv2.imread("src/utils/pitch_2.png")

# Get pitch dimensions
pitch_height, pitch_width, _ = pitch_img_original.shape

# Scale up the pitch image by a factor of 4
pitch_img_original = cv2.resize(pitch_img_original, (pitch_width * 4, pitch_height * 4))

# Get updated pitch dimensions after scaling
pitch_height, pitch_width, _ = pitch_img_original.shape

# Get frame dimensions from the video
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
if not ret:
    raise Exception("Could not read frame")
frame_height, frame_width = frame.shape[:2]
cap.release()

# Define colors for different roles (in BGR format for OpenCV)
role_colors = {
    "TEAM A": (255, 255, 255),  # white
    "TEAM B": (0, 0, 255),      # red
    "REF": (0, 255, 255),       # yellow
    "GK": (0, 0, 0)             # black
}

# Process each interesting frame
for interesting_frame in interesting_frames:
    # Create a fresh copy of the pitch for each frame
    pitch_img = pitch_img_original.copy()
    
    # Get the frame with detections
    frame_detections = next((item for item in detections if item["frame_id"] == interesting_frame), None)
    
    if frame_detections:
        for detection in frame_detections["detections"]:
            coords = detection["minimap_coordinates"]
            if coords is None:
                continue
            x = coords["x"]
            y = coords["y"]
            x_max = coords["x_max"]
            y_max = coords["y_max"]
            
            # Normalize to pitch dimensions (with scaling factor of 4 applied)
            norm_x = int(x / x_max * pitch_width)
            norm_y = int(y / y_max * pitch_height)
            
            # Get player role and determine color
            role = detection["role"] if "role" in detection else "UNKNOWN"
            color = role_colors.get(role, (255, 0, 0))  # default to blue
            
            # Draw player as a circle (increased radius to match the scaling)
            cv2.circle(pitch_img, (norm_x, norm_y), 60, color, -1)
    
    # Save the image
    minimap_output_path = f"{images_for_paper_path}/minimap_{interesting_frame:06d}.png"
    cv2.imwrite(minimap_output_path, pitch_img)
    print(f"Saved minimap visualization for frame {interesting_frame} to {minimap_output_path}")

Saved minimap visualization for frame 212 to data/example/images_for_paper/minimap_000212.png
Saved minimap visualization for frame 400 to data/example/images_for_paper/minimap_000400.png
Saved minimap visualization for frame 460 to data/example/images_for_paper/minimap_000460.png


In [82]:
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import cv2

# Process each interesting frame
for interesting_frame in interesting_frames:
    # Get the frame with detections
    frame_detections = next((item for item in detections if item["frame_id"] == interesting_frame), None)
    
    if not frame_detections:
        continue
    
    # Extract player positions (excluding referee)
    player_positions = []
    for detection in frame_detections["detections"]:
        coords = detection["minimap_coordinates"]
        role = detection["role"] if "role" in detection else "UNKNOWN"
        
        # Skip if coordinates are None or if it's a referee
        if coords is None or role == "REF":
            continue
        
        x = coords["x"]
        y = coords["y"]
        x_max = coords["x_max"]
        y_max = coords["y_max"]
        
        # Normalize to pitch dimensions (in meters)
        norm_x = x / x_max * 105  # 105m is pitch width
        norm_y = y / y_max * 68   # 68m is pitch height
        
        player_positions.append([norm_x, norm_y])

    # Convert to numpy array
    player_positions = np.array(player_positions)
    
    # Skip if no player positions
    if len(player_positions) == 0:
        continue
    
    # Run DBSCAN to identify clusters
    # eps=2 means players within 2 meters of each other
    # min_samples=2 means at least 2 players needed to form a cluster
    clustering = DBSCAN(eps=4, min_samples=2).fit(player_positions)
    
    # Get cluster labels
    labels = clustering.labels_
    
    # Number of clusters (excluding noise points with label -1)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    print(f"Frame {interesting_frame}: Number of clusters found: {n_clusters}")
    
    # Load the saved minimap image
    minimap_output_path = f"{images_for_paper_path}/minimap_{interesting_frame:06d}.png"
    minimap_img = cv2.imread(minimap_output_path)
    
    # For each cluster, draw a circle and the cluster center
    for cluster_id in range(n_clusters):
        # Get points in this cluster
        cluster_points = player_positions[labels == cluster_id]
        
        # Calculate cluster center
        center_x, center_y = np.mean(cluster_points, axis=0)
        
        # Convert center coordinates back to image pixels
        center_x_px = int(center_x / 105 * pitch_width)
        center_y_px = int(center_y / 68 * pitch_height)
        
        # Calculate the radius based on the spread of points in the cluster
        if len(cluster_points) > 1:
            # Calculate the average distance from points to center
            distances = np.sqrt(np.sum((cluster_points - np.array([center_x, center_y]))**2, axis=1))
            radius = int(np.max(distances) / 105 * pitch_width) + 80
            
            # Ensure minimum radius
            radius = max(radius, 20)
            
            # Draw circle around the cluster
            cv2.circle(minimap_img, 
                      (center_x_px, center_y_px), 
                      radius,
                      (0, 255, 0),  # Green color
                      8)  # Line thickness
        
        # Draw cluster center with green dot
        cv2.circle(minimap_img, (center_x_px, center_y_px), 20, (0, 255, 0), -1)  # Green dot
    
    # Save the image with clusters
    clusters_output_path = f"{images_for_paper_path}/minimap_decision_critical_zones_{interesting_frame:06d}.png"
    cv2.imwrite(clusters_output_path, minimap_img)
    print(f"Saved minimap with clusters for frame {interesting_frame} to {clusters_output_path}")


Frame 212: Number of clusters found: 6
Saved minimap with clusters for frame 212 to data/example/images_for_paper/minimap_decision_critical_zones_000212.png
Frame 400: Number of clusters found: 3
Saved minimap with clusters for frame 400 to data/example/images_for_paper/minimap_decision_critical_zones_000400.png
Frame 460: Number of clusters found: 2
Saved minimap with clusters for frame 460 to data/example/images_for_paper/minimap_decision_critical_zones_000460.png


In [87]:
# Create a figure showing player duels (players from opposing teams within 3 meters)
for interesting_frame in interesting_frames:
    # Get the frame with detections
    frame_detections = next((item for item in detections if item["frame_id"] == interesting_frame), None)
    
    if not frame_detections:
        continue
    
    # Extract player positions and team information
    player_positions = []
    player_teams = []
    referee_position = None  # Initialize referee position
    
    for detection in frame_detections["detections"]:
        coords = detection["minimap_coordinates"]
        role = detection["role"] if "role" in detection else "UNKNOWN"
        
        # Store referee position separately
        if role == "REF" and coords is not None:
            x = coords["x"]
            y = coords["y"]
            x_max = coords["x_max"]
            y_max = coords["y_max"]
            
            # Normalize to pitch dimensions (in meters)
            norm_x = x / x_max * 105  # 105m is pitch width
            norm_y = y / y_max * 68   # 68m is pitch height
            
            referee_position = np.array([norm_x, norm_y])
            continue
            
        # Skip if coordinates are None
        if coords is None:
            continue
        
        # Determine team based on role
        team = 1 if role == "TEAM A" else 2 if role == "TEAM B" else 0
        
        x = coords["x"]
        y = coords["y"]
        x_max = coords["x_max"]
        y_max = coords["y_max"]
        
        # Normalize to pitch dimensions (in meters)
        norm_x = x / x_max * 105  # 105m is pitch width
        norm_y = y / y_max * 68   # 68m is pitch height
        
        player_positions.append([norm_x, norm_y])
        player_teams.append(team)
    
    # Convert to numpy arrays
    player_positions = np.array(player_positions)
    player_teams = np.array(player_teams)
    
    # Skip if no player positions
    if len(player_positions) < 2:
        continue
    
    # Load the saved minimap image
    minimap_output_path = f"{images_for_paper_path}/minimap_{interesting_frame:06d}.png"
    minimap_img = cv2.imread(minimap_output_path)
    
    # Find duels (players from opposing teams within 3 meters)
    duels = []
    for i in range(len(player_positions)):
        for j in range(i+1, len(player_positions)):
            # Check if players are from different teams
            if player_teams[i] != player_teams[j]:
                # Calculate distance between players
                dist = np.sqrt(np.sum((player_positions[i] - player_positions[j])**2))
                
                # If distance is less than 3 meters, it's a duel
                if dist < 3:
                    duels.append((i, j))
    
    # Draw duels on the minimap
    for i, j in duels:
        # Get positions of the two players
        pos1, pos2 = player_positions[i], player_positions[j]
        
        # Convert coordinates to image pixels
        x1_px = int(pos1[0] / 105 * pitch_width)
        y1_px = int(pos1[1] / 68 * pitch_height)
        x2_px = int(pos2[0] / 105 * pitch_width)
        y2_px = int(pos2[1] / 68 * pitch_height)
        
        # Draw a line connecting the two players
        cv2.line(minimap_img, 
                (x1_px, y1_px), 
                (x2_px, y2_px), 
                (0, 255, 0),  # Green color
                8)  # Line thickness
        
        # Draw dots for the players
        team1_color = (0, 255, 0)  # Green for team 1
        team2_color = (0, 255, 0)  # Green for team 2
        
        cv2.circle(minimap_img, (x1_px, y1_px), 20, 
                  team1_color if player_teams[i] == 1 else team2_color, -1)
        cv2.circle(minimap_img, (x2_px, y2_px), 20, 
                  team1_color if player_teams[j] == 1 else team2_color, -1)
    
    # Draw lines from referee to the two closest duels if referee is present
    if referee_position is not None and duels:
        # Convert referee position to image pixels
        ref_x_px = int(referee_position[0] / 105 * pitch_width)
        ref_y_px = int(referee_position[1] / 68 * pitch_height)
        
        # Draw referee position
        cv2.circle(minimap_img, (ref_x_px, ref_y_px), 20, (0, 255, 0), -1)  # Green dot for referee
        
        # Calculate distances from referee to all duels
        duel_distances = []
        
        for idx, (i, j) in enumerate(duels):
            # Calculate midpoint of the duel
            duel_midpoint = (player_positions[i] + player_positions[j]) / 2
            
            # Calculate distance from referee to duel midpoint
            dist = np.sqrt(np.sum((referee_position - duel_midpoint)**2))
            
            duel_distances.append((idx, dist))
        
        # Sort duels by distance to referee
        duel_distances.sort(key=lambda x: x[1])
        
        # Draw lines to the two closest duels if available
        for k in range(min(2, len(duel_distances))):
            closest_duel_idx = duel_distances[k][0]
            i, j = duels[closest_duel_idx]
            
            # Get midpoint of the duel
            duel_midpoint = (player_positions[i] + player_positions[j]) / 2
            
            # Convert midpoint to image pixels
            mid_x_px = int(duel_midpoint[0] / 105 * pitch_width)
            mid_y_px = int(duel_midpoint[1] / 68 * pitch_height)
            
            # Draw a line from referee to duel midpoint
            cv2.line(minimap_img, 
                    (ref_x_px, ref_y_px), 
                    (mid_x_px, mid_y_px), 
                    (0, 255, 0),  # Green color
                    8,  # Line thickness
                    cv2.LINE_AA)  # Anti-aliased line
            
            # Calculate angle between duel line and referee line
            duel_vector = player_positions[j] - player_positions[i]
            ref_to_duel_vector = duel_midpoint - referee_position
            
            # Calculate unit vectors
            duel_unit = duel_vector / np.linalg.norm(duel_vector)
            ref_unit = ref_to_duel_vector / np.linalg.norm(ref_to_duel_vector)
            
            # Calculate dot product and angle
            dot_product = np.dot(duel_unit, ref_unit)
            # Clip to handle floating point errors
            dot_product = np.clip(dot_product, -1.0, 1.0)
            angle_rad = np.arccos(dot_product)
            angle_deg = np.degrees(angle_rad)
            
            # Always get the acute angle (less than 90 degrees)
            if angle_deg > 90:
                angle_deg = 180 - angle_deg
                
            print(f"Frame {interesting_frame}: Angle between duel {k+1} and referee line: {angle_deg:.2f} degrees")
    
    # Save the image with duels
    duels_output_path = f"{images_for_paper_path}/minimap_duels_{interesting_frame:06d}.png"
    cv2.imwrite(duels_output_path, minimap_img)
    print(f"Frame {interesting_frame}: Number of duels found: {len(duels)}")
    print(f"Saved minimap with duels for frame {interesting_frame} to {duels_output_path}")


Frame 212: Angle between duel 1 and referee line: 65.84 degrees
Frame 212: Angle between duel 2 and referee line: 35.79 degrees
Frame 212: Number of duels found: 5
Saved minimap with duels for frame 212 to data/example/images_for_paper/minimap_duels_000212.png
Frame 400: Angle between duel 1 and referee line: 48.95 degrees
Frame 400: Angle between duel 2 and referee line: 62.97 degrees
Frame 400: Number of duels found: 2
Saved minimap with duels for frame 400 to data/example/images_for_paper/minimap_duels_000400.png
Frame 460: Number of duels found: 0
Saved minimap with duels for frame 460 to data/example/images_for_paper/minimap_duels_000460.png
